In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency


In [2]:
df_ab = pd.read_csv('./b_a_test_cvr_file.csv')

In [3]:
df_ab.dropna(subset=['area_name'])

,area_name,variant,session_count,converts,Non-converts,CVR
0,Jahra - Saad Al Abdullah,Variation1,114151,5630,108521,4.93%
1,Jahra - Saad Al Abdullah,Control,108131,5502,102629,5.09%
2,Salmiya,Variation1,107102,7829,99273,7.31%
3,Salmiya,Control,106893,8383,98510,7.84%
4,Sabah Al-Salem,Control,86376,6388,79988,7.40%
...,...,...,...,...,...,...
272,Sheikh Saad Al Abdullah Airport,Control,129,10,119,7.75%
273,Wafra residential,Variation1,101,3,98,2.97%
274,South Abdullah Al Mubarak,Control,33,1,32,3.03%
275,South Abdullah Al Mubarak,Variation1,26,0,26,0.00%


In [4]:
var=df_ab['variant'].drop_duplicates()
print(var)

0    Variation1
1       Control
Name: variant, dtype: object


KeyError: '[5] not found in axis'

In [5]:
myorder = [ 1]
var = [var]
print(var)

[0    Variation1
1       Control
Name: variant, dtype: object]


In [6]:
df_ab=df_ab.sort_values(['session_count','area_name','variant'],ascending=False)
df_ab.head()

,area_name,variant,session_count,converts,Non-converts,CVR
0,Jahra - Saad Al Abdullah,Variation1,114151,5630,108521,4.93%
1,Jahra - Saad Al Abdullah,Control,108131,5502,102629,5.09%
2,Salmiya,Variation1,107102,7829,99273,7.31%
3,Salmiya,Control,106893,8383,98510,7.84%
4,Sabah Al-Salem,Control,86376,6388,79988,7.40%


In [7]:
df_ab.rename(columns = {'Non-converts':'n_converts'}, inplace = True)


In [8]:
#t=np.array([])
area=df_ab['area_name'].drop_duplicates()
area=area.dropna()
area
#for x in area:
   # for y in var:
      # print(x,df_ab['variant'].loc[df_ab['variant']==y])
     # print(x,df_ab.converts.loc[df_ab['variant']=='Control'], df_ab.n_converts.loc[df_ab['variant']==y])
     #   t.append(x)
      #  t= np.vstack(np.array([[df_ab.converts.loc[df_ab['variant']=='Control'],df_ab.n_converts.loc[df_ab['variant']=='Control']], [df_ab.converts.loc[df_ab['variant']==y] ,df_ab.n_converts.loc[df_ab['variant']==y]]]))
      #  print(t)

0             Jahra - Saad Al Abdullah
2                              Salmiya
4                       Sabah Al-Salem
6                                Salwa
7                       Jaber Al Ahmad
                    ...               
267                          Nuwaiseeb
270                  Wafra residential
271    Sheikh Saad Al Abdullah Airport
274          South Abdullah Al Mubarak
276                     Talabat Island
Name: area_name, Length: 138, dtype: object

In [9]:
df_ab.head(50)

,area_name,variant,session_count,converts,n_converts,CVR
0,Jahra - Saad Al Abdullah,Variation1,114151,5630,108521,4.93%
1,Jahra - Saad Al Abdullah,Control,108131,5502,102629,5.09%
2,Salmiya,Variation1,107102,7829,99273,7.31%
3,Salmiya,Control,106893,8383,98510,7.84%
4,Sabah Al-Salem,Control,86376,6388,79988,7.40%
5,Sabah Al-Salem,Variation1,86242,6272,79970,7.27%
6,Salwa,Variation1,78406,5156,73250,6.58%
7,Jaber Al Ahmad,Control,77558,3760,73798,4.85%
8,Jaber Al Ahmad,Variation1,75961,3646,72315,4.80%
9,Salwa,Control,75217,5285,69932,7.03%


In [37]:
T1 = np.array([[a.converts.loc[a.variant=='Control'], a.n_converts.loc[a.variant=='Control']], [a.converts.loc[a.variant==v], a.n_converts.loc[a.variant==v]]])
T1

C:\Users\elijah.paul\AppData\Local\Temp\ipykernel_13324\3859508673.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  T1 = np.array([[a.converts.loc[a.variant=='Control'], a.n_converts.loc[a.variant=='Control']], [a.converts.loc[a.variant==v], a.n_converts.loc[a.variant==v]]])


array([[1    5502
        Name: converts, dtype: int64, 1    102629
                                      Name: n_converts, dtype: int64],
       [0    5630
        1    5502
        Name: converts, dtype: int64, 0    108521
                                      1    102629
                                      Name: n_converts, dtype: int64]],
      dtype=object)

In [42]:
var

[0    Variation1
 1       Control
 Name: variant, dtype: object]

In [10]:
p_values = []
for ar in area:
    a=df_ab[df_ab['area_name']==ar]
    p_values=np.append(p_values,ar)
    T = np.array([[a.converts.loc[a.variant=='Control'], a.n_converts.loc[a.variant=='Control']], [a.converts.loc[a.variant=='Variation1'], a.n_converts.loc[a.variant=='Variation1']]])
    p1 = scipy.stats.chi2_contingency(T,correction=False)[1]
    p_values=np.append(p_values,p1)
      #  p_values.append(scipy.stats.chi2_contingency(T,correction=False)[1])
#table = list(ar, p_values)
#df2 = pd.DataFrame(table, columns=['test_name', 'p_value'])
        #df['kpi_label'] = column_name


C:\Users\elijah.paul\AppData\Local\Temp\ipykernel_18588\1143880354.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  T = np.array([[a.converts.loc[a.variant=='Control'], a.n_converts.loc[a.variant=='Control']], [a.converts.loc[a.variant=='Variation1'], a.n_converts.loc[a.variant=='Variation1']]])


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:

p_values

array(['Jahra - Saad Al Abdullah', '0.09147221496832754', 'Salmiya',
       '3.2368420732502026e-06', 'Sabah Al-Salem', '0.3269653346273659',
       'Salwa', '0.00045565477285427233', 'Jaber Al Ahmad',
       '0.6597710372131651', 'Abdullah Al-Mubarak - West Jeleeb',
       '0.00011829424107916458', 'Jabriya', '0.9438365104867813',
       'Sabahiya', '0.8137916330890145', 'Ardhiya', '0.878759475350681',
       'Hawally', '3.526886095506677e-05', 'Mubarak Al-Kabir',
       '2.0631801001407093e-06', 'Ferdous', '0.22116471682712308', 'Adan',
       '0.000561905525645992', 'Jaber Al Ali', '0.4021203805311486',
       'Abu Ftaira', '0.49869356664427633', 'Rumaithiya',
       '0.003228741877347287', 'Al-Qusour', '0.15689237305922532',
       'Sabah Al-Nasser', '0.00011317404502661605', 'Mangaf',
       '1.7259518678283258e-05', 'Andalous', '0.08988674870212124',
       'Bayan', '2.938798633646642e-06', 'Fnaitess', '0.2935473184857737',
       'Al-Qurain', '0.8190676153718583', 'Zahra', '0.26

In [17]:
#appending 0s to make the array reshapable. Ideally delete the last entry [0 implies significance.]
for i in range(1):
    p_values=p_values=np.append(p_values,0)

In [13]:
p_values

array(['Jahra - Saad Al Abdullah', '0.09147221496832754', 'Salmiya',
       '3.2368420732502026e-06', 'Sabah Al-Salem', '0.3269653346273659',
       'Salwa', '0.00045565477285427233', 'Jaber Al Ahmad',
       '0.6597710372131651', 'Abdullah Al-Mubarak - West Jeleeb',
       '0.00011829424107916458', 'Jabriya', '0.9438365104867813',
       'Sabahiya', '0.8137916330890145', 'Ardhiya', '0.878759475350681',
       'Hawally', '3.526886095506677e-05', 'Mubarak Al-Kabir',
       '2.0631801001407093e-06', 'Ferdous', '0.22116471682712308', 'Adan',
       '0.000561905525645992', 'Jaber Al Ali', '0.4021203805311486',
       'Abu Ftaira', '0.49869356664427633', 'Rumaithiya',
       '0.003228741877347287', 'Al-Qusour', '0.15689237305922532',
       'Sabah Al-Nasser', '0.00011317404502661605', 'Mangaf',
       '1.7259518678283258e-05', 'Andalous', '0.08988674870212124',
       'Bayan', '2.938798633646642e-06', 'Fnaitess', '0.2935473184857737',
       'Al-Qurain', '0.8190676153718583', 'Zahra', '0.26

In [15]:
p_values.shape()

TypeError: 'tuple' object is not callable

In [18]:
p_values=p_values.reshape(-1,2)
p_values

array([['Jahra - Saad Al Abdullah', '0.09147221496832754'],
       ['Salmiya', '3.2368420732502026e-06'],
       ['Sabah Al-Salem', '0.3269653346273659'],
       ['Salwa', '0.00045565477285427233'],
       ['Jaber Al Ahmad', '0.6597710372131651'],
       ['Abdullah Al-Mubarak - West Jeleeb', '0.00011829424107916458'],
       ['Jabriya', '0.9438365104867813'],
       ['Sabahiya', '0.8137916330890145'],
       ['Ardhiya', '0.878759475350681'],
       ['Hawally', '3.526886095506677e-05'],
       ['Mubarak Al-Kabir', '2.0631801001407093e-06'],
       ['Ferdous', '0.22116471682712308'],
       ['Adan', '0.000561905525645992'],
       ['Jaber Al Ali', '0.4021203805311486'],
       ['Abu Ftaira', '0.49869356664427633'],
       ['Rumaithiya', '0.003228741877347287'],
       ['Al-Qusour', '0.15689237305922532'],
       ['Sabah Al-Nasser', '0.00011317404502661605'],
       ['Mangaf', '1.7259518678283258e-05'],
       ['Andalous', '0.08988674870212124'],
       ['Bayan', '2.938798633646642e-06'],

In [19]:
df_results=pd.DataFrame(p_values,columns=['area','p_values'])

In [21]:
df_results.head(10)

,area,p_values
0,Jahra - Saad Al Abdullah,0.09147221496832754
1,Salmiya,3.2368420732502026e-06
2,Sabah Al-Salem,0.3269653346273659
3,Salwa,0.00045565477285427233
4,Jaber Al Ahmad,0.6597710372131651
5,Abdullah Al-Mubarak - West Jeleeb,0.00011829424107916458
6,Jabriya,0.9438365104867813
7,Sabahiya,0.8137916330890145
8,Ardhiya,0.878759475350681
9,Hawally,3.526886095506677e-05


In [22]:
convert_dict = {'area':object,
                'p_values':float,
                }
df_results = df_results.astype(convert_dict)

In [23]:
df_results.head()

,area,p_values
0,Jahra - Saad Al Abdullah,0.091472
1,Salmiya,0.000003
2,Sabah Al-Salem,0.326965
3,Salwa,0.000456
4,Jaber Al Ahmad,0.659771


In [70]:
#bonferroni's correction
df_results[df_results.select_dtypes(include=['number']).columns]*=5
df_results.head()

,area,Variation1,Variation2,Variation3,Variation4,Variation5
0,Jahra - Saad Al Abdullah,3.019436e-01,4.156211e-01,4.720292,0.012729,8.690435e-01
1,Salmiya,7.230401e-06,5.488949e-03,1.464952,1.488191,3.997295e-09
2,Sabah Al-Salem,9.286003e-10,8.611209e-07,0.002288,0.528827,1.190762e-01
3,Jaber Al Ahmad,3.811806e-01,1.772064e-02,0.049290,0.169643,3.687309e+00
4,Salwa,1.452394e-02,4.150246e-03,0.064208,0.350662,2.863110e-01


In [24]:
from pathlib import Path  
filepath = Path('./airflow_significance/chi2_values_ba_test.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
df_results.to_csv(filepath)  
#area.to_csv(index=True)